In [1]:
import os
%pwd

'c:\\Users\\Hung\\Desktop\\ML-Ops\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Hung\\Desktop\\ML-Ops'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# class to save configurations for data ingestion
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

# class to create dirs for the raw data
class ModelTrainerManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schemas = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schemas.name
        )
        
        return model_trainer_config

In [7]:
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import pandas as pd
import joblib

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig) -> None:
        self.config = config
        
    def train(self) -> bool:
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        logger.info("Train the model....")
        
        path=os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(lr, path)
        logger.info(f"Training was successful. Dump the model in [{path}]")

In [8]:
try:
    config = ModelTrainerManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-10-15 15:25:54,523: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-15 15:25:54,529: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-15 15:25:54,532: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-15 15:25:54,533: INFO: common: created directory at: artifacts]
[2023-10-15 15:25:54,534: INFO: common: created directory at: artifacts/model_trainer]
[2023-10-15 15:25:54,556: INFO: 2588956826: Training the model.]
[2023-10-15 15:25:54,558: INFO: 2588956826: Training was successful. Dump the model in [artifacts/model_trainer\model.joblib]]
